In [ ]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import collections
from matplotlib.pyplot import show
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 1
%aimport study_utils
%aimport lausanne_2016_utils

In [ ]:
PATH_TO_DATA = '../../Scraping/DataSport/Data/Lausanne_Marathon_2016.pickle'

# Data wrangling

In this first part, we retrieve data of Lausanne Marathon 2016 and we manipulate the different columns in order to exploit them during the analysis part.

First, we use the pickle format to allow faster import of the data, and correct formatting of some columns as date columns.

Note: Same data are also available in CSV (readable format).

In [ ]:
lausanne_marathon_2016 = pd.read_pickle(PATH_TO_DATA)
lausanne_marathon_2016.head()

Some participants are part of categories which must be excluded as these categories are not representative or can be misleading for the analysis part.

In [ ]:
lausanne_marathon_2016_cleaned = lausanne_marathon_2016[lausanne_marathon_2016.apply(study_utils.filter_participants, axis=1)].copy()

Here, we know the categories of the runners and these categories contains sex information about the participants (for a given type of running, we have two subcategories for male and female runners, as usual in such sports). Thus, we use this information to create a new column giving the sex of each participants.

In [ ]:
lausanne_marathon_2016_cleaned['sex'] = lausanne_marathon_2016_cleaned.apply(study_utils.get_sex_of_runner, axis=1)

In [ ]:
lausanne_marathon_2016_cleaned['sex'].value_counts(dropna=False)

In [ ]:
len(lausanne_marathon_2016_cleaned)

At the end of this stage, we have a total of 11248 runners on whom we can run our analysis for Lausanne Marathon 2016.

We also extract from the category the fact that a runner is either junior or adult.

In [ ]:
lausanne_marathon_2016_cleaned['type'] = lausanne_marathon_2016_cleaned.apply(study_utils.get_type_of_runner, axis=1)

We also transform the *number* and *rank* columns, as they contain integer.

In [ ]:
lausanne_marathon_2016_cleaned['number'] = lausanne_marathon_2016_cleaned['number'].apply(lambda x : int(float(x)))

In [ ]:
lausanne_marathon_2016_cleaned['rank'] = lausanne_marathon_2016_cleaned['rank'].apply(lambda x : int(float(x)))

In [ ]:
lausanne_marathon_2016_cleaned.head()

We also compute the age of runner based on the birthyear, to easily manipulate such information later.

In [ ]:
lausanne_marathon_2016_cleaned['age'] = lausanne_marathon_2016_cleaned.apply(study_utils.compute_age_of_runner, axis=1)
lausanne_marathon_2016_cleaned['age'] = lausanne_marathon_2016_cleaned['age'].apply(lambda x : int(float(x)))

In [ ]:
lausanne_marathon_2016_cleaned['age category'] = pd.cut(lausanne_marathon_2016_cleaned['age'], [15, 26, 31, 36, 41, 46, 51, 56, 61, 66, 100], labels=['15-25 years', '26-30 years', '31-35 years', '36-40 years', '41-45 years', '46-50 years', '51-55 years', '56-60 years', '61-65 years', '65+ years'], right=False)

If the time is in readable-format, it is difficult to use it this way for analysis as comparison can be misleading. For this reason, we decide to store time in raw format (i.e. number of seconds, without any format).

In [ ]:
lausanne_marathon_2016_cleaned['time'] = lausanne_marathon_2016_cleaned.apply(study_utils.format_time, axis=1)

Moreover, we store the type of running the participants did.

In [ ]:
lausanne_marathon_2016_cleaned['distance (km)'] = lausanne_marathon_2016_cleaned.apply(study_utils.compute_distance_from_category, axis=1)

We also need to compute the speed by kilometer for each runners.

In [ ]:
lausanne_marathon_2016_cleaned['speed (m/s)'] = lausanne_marathon_2016_cleaned['distance (km)']*1000/lausanne_marathon_2016_cleaned['time']
lausanne_marathon_2016_cleaned['profile'] = lausanne_marathon_2016_cleaned.apply(study_utils.compute_run_in_team, axis=1)

Finally, we make three distinct sets of data (10 km, semi-marathon and marathon).

In [ ]:
lausanne_marathon_2016_10km = lausanne_marathon_2016_cleaned[lausanne_marathon_2016_cleaned['distance (km)'] == 10]
lausanne_marathon_2016_21km = lausanne_marathon_2016_cleaned[lausanne_marathon_2016_cleaned['distance (km)'] == 21]
lausanne_marathon_2016_42km = lausanne_marathon_2016_cleaned[lausanne_marathon_2016_cleaned['distance (km)'] == 42]

In [ ]:
lausanne_marathon_2016_cleaned.to_pickle('./Data/Lausanne_Marathon_2016_cleaned.pickle')

# Complete study

## Overall study

In [ ]:
lausanne_2016_utils.plot_gender_distributions(lausanne_marathon_2016_cleaned)

As we can see, Lausanne Marathon race had a significantly higher percentage of male runners than female ones comparing to gender distribution of Canton of Vaud, which is equal regarding male and female.

In [ ]:
lausanne_2016_utils.plot_gender_distribution_according_to_running_type(lausanne_marathon_2016_10km, lausanne_marathon_2016_21km, lausanne_marathon_2016_42km)

We also see an interesting thing is that for 10 km running, distribution between male and female runners is similar, while for semi-marathon and marathon, we observe a clear difference between the repartition! According to the new direction of the Lausanne Marathon, there is a reason behind this observation.

> "Women don't even dare start the race unless they are totaly sure about mastering the distance, unlike men who sometimes finish completely exhausted.”
>
> Source: <a href="https://www.rts.ch/sport/athletisme/1258113-le-marathon-de-lausanne-change-dorientation.html">RTSSport (FR)</a>

In [ ]:
lausanne_2016_utils.plot_distribution_between_adults_and_juniors(lausanne_marathon_2016_cleaned)

Not surprinsingly, very minor part of runners were young runners in Lausanne Marathon of 2016. Indeed, these runnings ask a lot of effort and are very difficult for juniors runners.

## Demographic study

In this part, we plan to focus on age of runners, but also the place from which they come.

We first display the age distribution of runners in the marathon 2016 of Lausanne.

In [ ]:
lausanne_2016_utils.plot_age_distribution(lausanne_marathon_2016_cleaned)

We can notice that, globally, the average age of runners is 39 years. We also observe the conclusion we previously made (i.e. male runners are younger than female ones as women wait before trying to participate to a semi-marathon or a marathon).

In [ ]:
fig = plt.figure()
fig.tight_layout
fig.set_size_inches(10, 10)
fig.suptitle('Age distribution of runners according to the distance', fontsize=12)
lausanne_2016_utils.plot_distribution_age_distance(fig, lausanne_marathon_2016_10km, '10 km', 311)
lausanne_2016_utils.plot_distribution_age_distance(fig, lausanne_marathon_2016_21km, '21 km', 312)
lausanne_2016_utils.plot_distribution_age_distance(fig, lausanne_marathon_2016_42km, '42 km', 313)

Here, the important fact is that average age of runners tends to increase when number of kilometers of a runnings is increasing. This can be explained by the fact that it takes time before mastering such races like semi-marathons and marathons.

### Performance according to age of participants

#### Overall analysis

In [ ]:
lausanne_2016_utils.plot_time_distribution_by_age(lausanne_marathon_2016_cleaned, 'age category')

#### Performance for 10 km running

In [ ]:
lausanne_2016_utils.plot_performance_according_to_running_type(lausanne_marathon_2016_10km, 10, 'age', size=10, aspect=1.5)

In [ ]:
lausanne_2016_utils.plot_performance_according_to_running_type(lausanne_marathon_2016_10km, 10, 'age category')

#### Performance for demi-marathon

In [ ]:
lausanne_2016_utils.plot_performance_according_to_running_type(lausanne_marathon_2016_21km, 21, 'age', size=10, aspect=1.5)

In [ ]:
lausanne_2016_utils.plot_performance_according_to_running_type(lausanne_marathon_2016_21km, 21, 'age category')

#### Performance for marathon

In [ ]:
lausanne_2016_utils.plot_performance_according_to_running_type(lausanne_marathon_2016_42km, 42, 'age', size=10, aspect=1.5)

In [ ]:
lausanne_2016_utils.plot_performance_according_to_running_type(lausanne_marathon_2016_42km, 42, 'age category')

#### Analysis of results

Globally, we observe more real differences at the extremes. But are these differences significant? We decide to compute ANOVA and Tukey HSD to verify it.

In [ ]:
# TODO
#results_10km = study_utils.compute_anova_and_tukey_hsd(lausanne_marathon_2016_10km, 'age category', 'time')
#print('F value: ' + str(results_10km['f_value']) + '\nP value: ' + str(results_10km['p_value']) + '\n')
#results_10km['tukey_hsd'].rename(columns=lambda x: x.strip(), inplace=True)
#results_10km['tukey_hsd'].set_index(['group1', 'group2'])

In [ ]:
results_21km = study_utils.compute_anova_and_tukey_hsd(lausanne_marathon_2016_21km, 'age category', 'time')
print('F value: ' + str(results_21km['f_value']) + '\nP value: ' + str(results_21km['p_value']) + '\n')
results_21km['tukey_hsd'].rename(columns=lambda x: x.strip(), inplace=True)
results_21km['tukey_hsd'].set_index(['group1', 'group2'])

In [ ]:
results_42km = study_utils.compute_anova_and_tukey_hsd(lausanne_marathon_2016_42km, 'age category', 'time')
print('F value: ' + str(results_42km['f_value']) + '\nP value: ' + str(results_42km['p_value']) + '\n')
results_42km['tukey_hsd'].rename(columns=lambda x: x.strip(), inplace=True)
results_42km['tukey_hsd'].set_index(['group1', 'group2'])

In all the cases, p value is extremly low. This means that we can reject null hypothesis H0 and conclude safely that there are significant differences between age categories!

## Effective performance according to BIB number

In [ ]:
lausanne_2016_utils.plot_time_distribution_by_bib_numbers(lausanne_marathon_2016_cleaned)

We can recognize three major patterns following the BIB number of participants. Indeed, for each subdivisions, the higher the BIB number is, the lower the performance time is, globally.

Notice that BIB numbers are given by the organizators of Lausanne Marathon, and as we can read on <a href="http://fr.lausanne-marathon.com/inscription/inscriptions/inscription-online/">the official website</a>, each participant is asked to indicate his "estimated time of running for the attribution of start blocks", in oder to categorize the runners accordingly.

## Study by sex

In [ ]:
# Creation of figure
fig = plt.figure()
fig.tight_layout
fig.set_size_inches(10, 10)
fig.suptitle('Speed distribution selected by distance', fontsize=14)
lausanne_2016_utils.plot_speed_distribution_by_running(fig, lausanne_marathon_2016_10km, '10 km', 1, np.arange(0,900, 100))
lausanne_2016_utils.plot_speed_distribution_by_running(fig, lausanne_marathon_2016_21km, '21 km', 2, np.arange(0,700, 100))
lausanne_2016_utils.plot_speed_distribution_by_running(fig, lausanne_marathon_2016_42km, '42 km', 3, np.arange(0,250, 50))

As we can expect the fastest runners have made the 10 km, as the distance is shorter, the speed by kilometer can be greater.

But unexpectedly the average speed seems very close between runners despite the difference of the distance, in order to understand this phenomenon we need to go further in the study and study the past of runners.

### Statistical analysis

** Overview **

In [ ]:
plot = sns.factorplot(data=lausanne_marathon_2016_cleaned, x='sex', y='speed (m/s)', col = 'distance (km)', size=5, kind='box')
plot.set_xlabels('Gender')
plot.set_ylabels('Speed (m/s)')
titles = ['10 km', 'Semi-marathon', 'Marathon']
for ax, title in zip(plot.axes.flat, titles):
    ax.set_title(title)
    ax.set(xticklabels=['Female', 'Male'])
plot.fig.set_size_inches(10, 6)

Following the boxplots, differences seem to be significant.

** 10 km running **

In [ ]:
results_10km = study_utils.compute_anova_and_tukey_hsd(lausanne_marathon_2016_10km, 'sex', 'speed (m/s)')
print('F value: ' + str(results_10km['f_value']) + '\nP value: ' + str(results_10km['p_value']) + '\n')
results_10km['tukey_hsd'].rename(columns=lambda x: x.strip(), inplace=True)
results_10km['tukey_hsd'].set_index(['group1', 'group2'])

** Semi-marathon **

In [ ]:
results_21km = study_utils.compute_anova_and_tukey_hsd(lausanne_marathon_2016_21km, 'sex', 'speed (m/s)')
print('F value: ' + str(results_21km['f_value']) + '\nP value: ' + str(results_21km['p_value']) + '\n')
results_21km['tukey_hsd'].rename(columns=lambda x: x.strip(), inplace=True)
results_21km['tukey_hsd'].set_index(['group1', 'group2'])

** Marathon **

In [ ]:
results_42km = study_utils.compute_anova_and_tukey_hsd(lausanne_marathon_2016_42km, 'sex', 'speed (m/s)')
print('F value: ' + str(results_42km['f_value']) + '\nP value: ' + str(results_42km['p_value']) + '\n')
results_42km['tukey_hsd'].rename(columns=lambda x: x.strip(), inplace=True)
results_42km['tukey_hsd'].set_index(['group1', 'group2'])

As we can see, no matter the running we consider, the difference between male and female is significant as we can reject H0 hypothesis. Notice that p-value is higher for the case of marathon, as the distribution seem to be more similar (see previous plots).

## Focus on teams and single runners

First, let's display the distribution of runners with and without teams.

In [ ]:
total_10 = len(lausanne_marathon_2016_10km)
total_21 = len(lausanne_marathon_2016_21km)
total_42 = len(lausanne_marathon_2016_42km)
lausanne_2016_utils.plot_distribution_team_individuals(lausanne_marathon_2016_cleaned, total_10, total_21, total_42)

The repartition individual/team seems quite equivalent between the different races offered by the event.
The teams are more present in the 10 km. The distance seems more appropriate for the teams, it's short enough to be a good challenge between beginner friends.

### Statistical analysis

Let's see if being in team has a real impact on the performance.

In [ ]:
plot = sns.factorplot(data=lausanne_marathon_2016_cleaned, x='profile', y='speed (m/s)', col = 'distance (km)', kind='box')
plot.fig.suptitle('Speed distribution by types of runners', fontsize=14)
titles = ['10 km', 'Semi-marathon', 'Marathon']
for ax, title in zip(plot.axes.flat, titles):
    ax.set_title(title)
    ax.set(xticklabels=['Individual runners', 'Team-mate runners'])
plot.set_xlabels('')
plot.set_ylabels('Speed (m/s)')
plot.fig.set_size_inches(10,6)

In [ ]:
lausanne_2016_utils.display_information_speed(lausanne_marathon_2016_cleaned)

Actually the teams are better on average compared to individual runners, but the best performance comes from individual runners.
From the graph above, we can notice that for "average" people, to be part of a team seems to be a motivational factor, compared to excellent runners who prefer doing the race alone.

Running ANOVA permits us to draw the same conclusion, as showed in the following results. Notice that for the marathon, p value is higher than for the other races as medians tend to be similar between the two categories (runners with and without team), even if it is not sufficiently high to maintain H0 hypothesis (i.e. similar distributions).

In [ ]:
results_teams_10km = study_utils.compute_anova_and_tukey_hsd(lausanne_marathon_2016_10km, 'profile', 'speed (m/s)')
print('F value: ' + str(results_teams_10km['f_value']) + '\nP value: ' + str(results_teams_10km['p_value']) + '\n')
results_teams_10km['tukey_hsd'].rename(columns=lambda x: x.strip(), inplace=True)
results_teams_10km['tukey_hsd'].set_index(['group1', 'group2'])

In [ ]:
results_teams_21km = study_utils.compute_anova_and_tukey_hsd(lausanne_marathon_2016_21km, 'profile', 'speed (m/s)')
print('F value: ' + str(results_teams_21km['f_value']) + '\nP value: ' + str(results_teams_21km['p_value']) + '\n')
results_teams_21km['tukey_hsd'].rename(columns=lambda x: x.strip(), inplace=True)
results_teams_21km['tukey_hsd'].set_index(['group1', 'group2'])

In [ ]:
results_teams_42km = study_utils.compute_anova_and_tukey_hsd(lausanne_marathon_2016_42km, 'profile', 'speed (m/s)')
print('F value: ' + str(results_teams_42km['f_value']) + '\nP value: ' + str(results_teams_42km['p_value']) + '\n')
results_teams_42km['tukey_hsd'].rename(columns=lambda x: x.strip(), inplace=True)
results_teams_42km['tukey_hsd'].set_index(['group1', 'group2'])

In [ ]:
team_selected = lausanne_marathon_2016_cleaned[lausanne_marathon_2016_cleaned['profile'] == 'Team-mate']
lausanne_marathon_2016_cleaned['time difference team'] = lausanne_marathon_2016_cleaned.apply(study_utils.compute_time_to_best_in_team,args=(team_selected,), axis=1)

In [ ]:
lausanne_marathon_team = lausanne_marathon_2016_cleaned[lausanne_marathon_2016_cleaned['profile'] == 'Team-mate'].copy()
lausanne_2016_utils.plot_time_difference_distribution(lausanne_marathon_team)

We observe that when runners are in team, they globally share similar performance (time's difference is between 0 and 5 minutes for the majority of runners). This indicates that similar people compose each team and they ran together, as we can expect it.

In [ ]:
fig = plt.figure()
fig.tight_layout
fig.set_size_inches(15, 15)
fig.suptitle('Team representation', fontsize=14)
annotation_plot1 = [311,'Team-mates',(0,2000), (0,2500)]
annotation_plot2 = [313,'Individual runner',(2,4000), (2,6000)]
lausanne_2016_utils.plot_scatter_difference_time_number(fig, lausanne_marathon_2016_cleaned, 10, 311, annotation_plot1)
lausanne_2016_utils.plot_scatter_difference_time_number(fig, lausanne_marathon_2016_cleaned, 21, 312)
lausanne_2016_utils.plot_scatter_difference_time_number(fig, lausanne_marathon_2016_cleaned, 42, 313, annotation_plot2)

As we can noticed from the previous study most of pair/group of runner have finished the race together. In order to go futher we need to study the situation of those who finihsed later. (study above take runners wither a minimum late of 1000 seconds after the best runners in the team).

We have considered a pair/group runner if the distance time if weaker than 30sec. It is very  unlikely that any two runners would have the exact same start and end time (within 30 sec) unless they ran together, so these visual clusters should be very accurate in showing running pairs/groups.Based on the graph there are distinct differences in "belong to a team" in the difference race, Based on the graph the 10km seems to be the more solidary one, where most people in a team ran in pair/group (326 runners compared to 60 runners who have finished alone). The 21km had a half of runner who have ran in pair/group whereas the 42km seems the more individiual one where 51 runners finished alone compared to 17 runners in teams/groups. 42 km is a long distance, the challenge is often individual and often seen as personnal reward of long hour of training, this could explain the difference in type of runners belong to a team.